## Getting the Grid

To begin, we need a spectral model library that we will use for our fitting. One common example are the PHOENIX models, most recently computed by T.O. Husser. We provide many interfaces directly with different libraries, which can be viewed in [Raw Grid Interfaces](api/grid_tools.rst#raw-grid-interfaces).

As a convenience, we provide a helper to download PHOENIX models from the Goettingen servers

In [1]:
import itertools
import numpy as np

from Starfish.grid_tools import download_PHOENIX_models

T = np.hstack([np.arange(5900, 7000, 100), np.arange(7000, 7500, 200)])
logg = np.arange(3.0, 6.01, 0.5)
Z = np.arange(-1, 1.01, 0.5)
params = list(itertools.product(T, logg, Z))

download_PHOENIX_models(path='PHOENIX', parameters=params)

lte07400-6.00+1.0.PHOENIX-ACES-AGSS-COND-2011-HiRes.fits: 100%|██████████| 490/490 [26:09<00:00,  4.80s/it]


Now that we have the files downloaded, let's set up a grid interface

In [2]:
from Starfish.grid_tools import PHOENIXGridInterfaceNoAlpha

grid = PHOENIXGridInterfaceNoAlpha(path='PHOENIX')

From here, we will want to set up our HDF5 interface that will allow us to go on to using the spectral emulator, but first we need to determine our model subset and instrument.

## Setting up the HDF5 Interface

We set up an HDF5 interface in order to allow much quicker reading and writing than compared to loading FITS files over and over again. In addition, when considering the application to our likelihood methods, we know that for a given dataset, any effects characteristic of the instrument can be pre-applied to our models, saving on computation time during the maximum likelihood estimation.

Looking towards our fitting examples, we know we will try fitting some data from the TRES spectrograph. We provide many popular spectrographs in our grid tools, including TRES.

Let’s also say that, for a given dataset (in our future examples we use WASP 14 so let’s consider that), we want to only use a reasonable subset of our original model grid. WASP 14 is currently labeled as an F5V star, so let’s create a subset around that classification.

In [3]:
from Starfish.grid_tools.instruments import TRES
from Starfish.grid_tools import HDF5Creator

ranges = [
    [5900, 7400],
    [3.0, 6.0],
    [-1.0, 1.0]
]
creator = HDF5Creator(grid, 'F_TRES_grid.hdf5', instrument=TRES(), ranges=ranges)
creator.process_grid()

Processing [7.4e+03 6.0e+00 1.0e+00]: 100%|██████████| 490/490 [08:02<00:00,  1.00it/s]


## Setting up the Spectral Emulator

Once we have our pre-processed grid, we can make our spectral emulator and train its Gaussian process hyperparameters.

In [4]:
from Starfish.grid_tools import HDF5Interface
from Starfish.emulator import Emulator

grid = HDF5Interface('F_TRES_grid.hdf5')
emu = Emulator.from_grid(grid)
emu

Emulator
--------
Trained: False
lambda_xi: 2.718
Variances:
	10000.00
	10000.00
	10000.00
	10000.00
	10000.00
	10000.00
	10000.00
Lengthscales:
	[ 600.00  1.50  1.50 ]
	[ 600.00  1.50  1.50 ]
	[ 600.00  1.50  1.50 ]
	[ 600.00  1.50  1.50 ]
	[ 600.00  1.50  1.50 ]
	[ 600.00  1.50  1.50 ]
	[ 600.00  1.50  1.50 ]
Log Likelihood: -761460.31

In [ ]:
%time emu.train()
emu

<div class"alert alert-warning">

**Warning:** Training the emulator will take on the order of minutes to complete. The more eigenspectra that are used as well as the resolution of the spectrograph will mainly dominate this runtime.

</div>